# Лабораторная работа №1: KNN

1. Бейзлайн sklearn (KNeighborsClassifier, KNeighborsRegressor)
2. Улучшение через подбор гиперпараметров
3. Собственная имплементация

## Импорт и загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import openml
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')
print("Импорт завершён")

Импорт завершён


### Датасет классификации: APS Failure at Scania Trucks
Бинарная классификация неисправности системы давления воздуха.

In [2]:
# Загрузка классификации
dataset = openml.datasets.get_dataset(41138)
X_clf, y_clf, _, _ = dataset.get_data(target=dataset.default_target_attribute)
y_clf_enc = (y_clf == 'pos').astype(int)

imputer = SimpleImputer(strategy='median')
X_clf_imp = pd.DataFrame(imputer.fit_transform(X_clf), columns=X_clf.columns)

# Сэмплируем для скорости KNN
np.random.seed(42)
idx = np.random.choice(len(X_clf_imp), 10000, replace=False)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf_imp.iloc[idx], y_clf_enc.iloc[idx], test_size=0.2, random_state=42, stratify=y_clf_enc.iloc[idx]
)
print(f"Классификация: train={X_clf_train.shape}, test={X_clf_test.shape}")
print(f"Доля положительного класса: {y_clf_train.mean():.4f}")

Классификация: train=(8000, 170), test=(2000, 170)
Доля положительного класса: 0.0189


### Датасет регрессии: Avocado Prices
Предсказание средней цены авокадо на основе объёма продаж и региона.

In [3]:
# Загрузка регрессии: Avocado Prices
path = kagglehub.dataset_download("neuromusic/avocado-prices")
df = pd.read_csv(os.path.join(path, "avocado.csv"))

# Предобработка
df['type_enc'] = LabelEncoder().fit_transform(df['type'])
df['region_enc'] = LabelEncoder().fit_transform(df['region'])

features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'year', 'type_enc', 'region_enc']
X_reg = df[features].values
y_reg = df['AveragePrice'].values

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)
print(f"Регрессия: train={X_reg_train.shape}, test={X_reg_test.shape}")
print(f"Цена: min={y_reg.min():.2f}, max={y_reg.max():.2f}, mean={y_reg.mean():.2f}")

Регрессия: train=(14599, 8), test=(3650, 8)
Цена: min=0.44, max=3.25, mean=1.41


## 2. Бейзлайн

Обучаем базовую модель KNeighborsClassifier из sklearn с параметрами по умолчанию (k=5). Оцениваем качество по F1-score и ROC-AUC на тестовой выборке.

In [4]:
# Бейзлайн классификация
knn_clf_base = KNeighborsClassifier(n_neighbors=5)
knn_clf_base.fit(X_clf_train, y_clf_train)
y_pred_base = knn_clf_base.predict(X_clf_test)
y_proba_base = knn_clf_base.predict_proba(X_clf_test)[:, 1]

f1_base = f1_score(y_clf_test, y_pred_base)
roc_base = roc_auc_score(y_clf_test, y_proba_base)

print(f"=== БЕЙЗЛАЙН: KNN (k=5) ===")
print(f"F1: {f1_base:.4f}, ROC-AUC: {roc_base:.4f}")

=== БЕЙЗЛАЙН: KNN (k=5) ===
F1: 0.3214, ROC-AUC: 0.8577


Обучаем KNeighborsRegressor с параметрами по умолчанию (k=5). Оцениваем качество по RMSE и R² на тестовой выборке.

In [5]:
# Бейзлайн регрессия
knn_reg_base = KNeighborsRegressor(n_neighbors=5)
knn_reg_base.fit(X_reg_train, y_reg_train)
y_pred_reg_base = knn_reg_base.predict(X_reg_test)

rmse_base = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_base))
r2_base = r2_score(y_reg_test, y_pred_reg_base)

print(f"=== БЕЙЗЛАЙН: KNN Regressor ===")
print(f"RMSE: {rmse_base:.4f}, R²: {r2_base:.4f}")

=== БЕЙЗЛАЙН: KNN Regressor ===
RMSE: 0.2745, R²: 0.5311


## 3. Улучшение бейзлайна

### Гипотезы:
- Масштабирование признаков (StandardScaler)
- Подбор k и weights

Применяем StandardScaler для нормализации признаков. Это критически важно для KNN, так как алгоритм использует расстояния между точками. Без масштабирования признаки с большими значениями будут доминировать.

In [6]:
# Масштабирование
scaler_clf = StandardScaler()
X_clf_train_sc = scaler_clf.fit_transform(X_clf_train)
X_clf_test_sc = scaler_clf.transform(X_clf_test)

scaler_reg = StandardScaler()
X_reg_train_sc = scaler_reg.fit_transform(X_reg_train)
X_reg_test_sc = scaler_reg.transform(X_reg_test)

# Бейзлайн НА МАСШТАБИРОВАННЫХ данных (для честного сравнения)
knn_clf_sc = KNeighborsClassifier(n_neighbors=5)
knn_clf_sc.fit(X_clf_train_sc, y_clf_train)
y_pred_sc = knn_clf_sc.predict(X_clf_test_sc)
y_proba_sc = knn_clf_sc.predict_proba(X_clf_test_sc)[:, 1]
f1_scaled = f1_score(y_clf_test, y_pred_sc)
roc_scaled = roc_auc_score(y_clf_test, y_proba_sc)

knn_reg_sc = KNeighborsRegressor(n_neighbors=5)
knn_reg_sc.fit(X_reg_train_sc, y_reg_train)
y_pred_reg_sc = knn_reg_sc.predict(X_reg_test_sc)
rmse_scaled = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_sc))
r2_scaled = r2_score(y_reg_test, y_pred_reg_sc)

print("=== ЭФФЕКТ МАСШТАБИРОВАНИЯ (k=5, без подбора) ===")
print(f"Классификация: F1 {f1_base:.4f} -> {f1_scaled:.4f} ({f1_scaled-f1_base:+.4f})")
print(f"Регрессия: R² {r2_base:.4f} -> {r2_scaled:.4f} ({r2_scaled-r2_base:+.4f})")

=== ЭФФЕКТ МАСШТАБИРОВАНИЯ (k=5, без подбора) ===
Классификация: F1 0.3214 -> 0.4528 (+0.1314)
Регрессия: R² 0.5311 -> 0.8461 (+0.3150)


Подбор гиперпараметров для классификации методом GridSearchCV с 3-fold кросс-валидацией. Подбираем количество соседей `k` и способ взвешивания (`uniform`/`distance`). Оптимизируем по метрике F1.

In [7]:
# GridSearch для классификации
param_grid = {'n_neighbors': [3, 5, 7, 11], 'weights': ['uniform', 'distance']}
grid_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_clf.fit(X_clf_train_sc, y_clf_train)

print(f"Лучшие параметры: {grid_clf.best_params_}")
y_pred_imp = grid_clf.predict(X_clf_test_sc)
y_proba_imp = grid_clf.predict_proba(X_clf_test_sc)[:, 1]

f1_imp = f1_score(y_clf_test, y_pred_imp)
roc_imp = roc_auc_score(y_clf_test, y_proba_imp)

print(f"=== УЛУЧШЕННЫЙ: KNN (подбор гиперпараметров) ===")
print(f"F1: {f1_imp:.4f} (vs scaled baseline {f1_scaled:.4f}: {f1_imp-f1_scaled:+.4f})")
print(f"ROC-AUC: {roc_imp:.4f} (vs scaled baseline {roc_scaled:.4f}: {roc_imp-roc_scaled:+.4f})")

Лучшие параметры: {'n_neighbors': 3, 'weights': 'uniform'}
=== УЛУЧШЕННЫЙ: KNN (подбор гиперпараметров) ===
F1: 0.4839 (vs scaled baseline 0.4528: +0.0310)
ROC-AUC: 0.8242 (vs scaled baseline 0.9030: -0.0788)


Подбор гиперпараметров для регрессии методом GridSearchCV. Оптимизируем по метрике R². Сравниваем улучшение RMSE и R² относительно бейзлайна.

In [8]:
# GridSearch для регрессии
grid_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_reg_train_sc, y_reg_train)

print(f"Лучшие параметры: {grid_reg.best_params_}")
y_pred_reg_imp = grid_reg.predict(X_reg_test_sc)

rmse_imp = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_imp))
r2_imp = r2_score(y_reg_test, y_pred_reg_imp)

print(f"=== УЛУЧШЕННЫЙ (подбор гиперпараметров) ===")
print(f"RMSE: {rmse_imp:.4f} (vs scaled baseline {rmse_scaled:.4f}: {rmse_imp-rmse_scaled:+.4f})")
print(f"R²: {r2_imp:.4f} (vs scaled baseline {r2_scaled:.4f}: {r2_imp-r2_scaled:+.4f})")

Лучшие параметры: {'n_neighbors': 5, 'weights': 'distance'}
=== УЛУЧШЕННЫЙ (подбор гиперпараметров) ===
RMSE: 0.1516 (vs scaled baseline 0.1572: -0.0056)
R²: 0.8569 (vs scaled baseline 0.8461: +0.0108)


## 4. Собственная имплементация

### 4a. Реализация KNN

Реализуем алгоритм KNN с нуля. Класс `MyKNN` поддерживает:
- Классификацию и регрессию (параметр `task`)
- Взвешивание по расстоянию (`weights='distance'`)
- Евклидово расстояние для поиска соседей
- Метод `predict_proba` для оценки вероятностей классов

In [9]:
class MyKNN:
    """KNN для классификации и регрессии"""
    
    def __init__(self, n_neighbors=5, weights='uniform', task='classification'):
        self.k = n_neighbors
        self.weights = weights
        self.task = task
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        return self
    
    def _get_neighbors(self, x):
        distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        indices = np.argsort(distances)[:self.k]
        return indices, distances[indices]
    
    def _predict_one(self, x):
        indices, distances = self._get_neighbors(x)
        neighbors_y = self.y_train[indices]
        
        if self.weights == 'distance':
            w = 1 / (distances + 1e-10)
        else:
            w = np.ones(self.k)
        
        if self.task == 'classification':
            votes = np.bincount(neighbors_y.astype(int), weights=w, minlength=2)
            return np.argmax(votes)
        else:
            return np.average(neighbors_y, weights=w)
    
    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_one(x) for x in X])
    
    def predict_proba(self, X):
        X = np.array(X)
        probs = []
        for x in X:
            indices, distances = self._get_neighbors(x)
            neighbors_y = self.y_train[indices]
            if self.weights == 'distance':
                w = 1 / (distances + 1e-10)
            else:
                w = np.ones(self.k)
            p1 = np.sum(w[neighbors_y == 1]) / np.sum(w)
            probs.append([1 - p1, p1])
        return np.array(probs)

print("MyKNN реализован!")

MyKNN реализован!


### 4b-d. Обучение и оценка

In [10]:
# Своя классификация (на меньшей выборке для скорости)
np.random.seed(42)
idx_small = np.random.choice(len(X_clf_train_sc), 2000, replace=False)

my_knn_clf = MyKNN(n_neighbors=5, task='classification')
my_knn_clf.fit(X_clf_train_sc[idx_small], y_clf_train.values[idx_small])
my_pred = my_knn_clf.predict(X_clf_test_sc)
my_proba = my_knn_clf.predict_proba(X_clf_test_sc)[:, 1]

my_f1 = f1_score(y_clf_test, my_pred)
my_roc = roc_auc_score(y_clf_test, my_proba)

print(f"=== СВОЯ РЕАЛИЗАЦИЯ: Классификация ===")
print(f"F1: {my_f1:.4f} (sklearn: {f1_base:.4f})")
print(f"ROC-AUC: {my_roc:.4f}")

=== СВОЯ РЕАЛИЗАЦИЯ: Классификация ===
F1: 0.1463 (sklearn: 0.3214)
ROC-AUC: 0.8253


Тестируем собственную реализацию KNN на задаче регрессии. Используем подвыборку для ускорения обучения. Сравниваем результаты со sklearn-бейзлайном.

In [11]:
# Своя регрессия
idx_reg = np.random.choice(len(X_reg_train_sc), 3000, replace=False)

my_knn_reg = MyKNN(n_neighbors=5, task='regression')
my_knn_reg.fit(X_reg_train_sc[idx_reg], y_reg_train[idx_reg])
my_pred_reg = my_knn_reg.predict(X_reg_test_sc)

my_rmse = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg))
my_r2 = r2_score(y_reg_test, my_pred_reg)

print(f"=== СВОЯ РЕАЛИЗАЦИЯ: Регрессия ===")
print(f"RMSE: {my_rmse:.4f} (sklearn: {rmse_base:.4f})")
print(f"R²: {my_r2:.4f} (sklearn: {r2_base:.4f})")

=== СВОЯ РЕАЛИЗАЦИЯ: Регрессия ===
RMSE: 0.2169 (sklearn: 0.2745)
R²: 0.7072 (sklearn: 0.5311)


### 4f-i. Улучшенная своя реализация

In [12]:
# Улучшенная своя (weights='distance')
best_k = grid_clf.best_params_['n_neighbors']
my_knn_clf_imp = MyKNN(n_neighbors=best_k, weights='distance', task='classification')
my_knn_clf_imp.fit(X_clf_train_sc[idx_small], y_clf_train.values[idx_small])
my_pred_imp = my_knn_clf_imp.predict(X_clf_test_sc)
my_proba_imp = my_knn_clf_imp.predict_proba(X_clf_test_sc)[:, 1]

my_f1_imp = f1_score(y_clf_test, my_pred_imp)
my_roc_imp = roc_auc_score(y_clf_test, my_proba_imp)

print(f"=== УЛУЧШЕННАЯ СВОЯ ===")
print(f"F1: {my_f1_imp:.4f}, ROC-AUC: {my_roc_imp:.4f}")

=== УЛУЧШЕННАЯ СВОЯ ===
F1: 0.3333, ROC-AUC: 0.7741


Применяем лучшие найденные гиперпараметры для регрессии с использованием взвешивания по расстоянию (`weights='distance'`). Это позволяет более близким соседям вносить больший вклад в предсказание.

In [13]:
# Улучшенная регрессия
best_k_reg = grid_reg.best_params_['n_neighbors']
my_knn_reg_imp = MyKNN(n_neighbors=best_k_reg, weights='distance', task='regression')
my_knn_reg_imp.fit(X_reg_train_sc[idx_reg], y_reg_train[idx_reg])
my_pred_reg_imp = my_knn_reg_imp.predict(X_reg_test_sc)

my_rmse_imp = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg_imp))
my_r2_imp = r2_score(y_reg_test, my_pred_reg_imp)

print(f"=== УЛУЧШЕННАЯ СВОЯ: Регрессия ===")
print(f"RMSE: {my_rmse_imp:.4f}, R²: {my_r2_imp:.4f}")

=== УЛУЧШЕННАЯ СВОЯ: Регрессия ===
RMSE: 0.1998, R²: 0.7516


## Итоговая сводка

In [14]:
print("="*70)
print("ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №1 (KNN)")
print("="*70)

print(f"\n{'КЛАССИФИКАЦИЯ (APS Failure)':-^70}")
print(f"{'Модель':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Бейзлайн (без масштаб.)':<30} {f1_base:<12.4f} {roc_base:<12.4f}")
print(f"{'Бейзлайн + масштабирование':<30} {f1_scaled:<12.4f} {roc_scaled:<12.4f}")
print(f"{'+ подбор гиперпараметров':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_f1:<12.4f} {my_roc:<12.4f}")
print(f"{'Своя улучшенная':<30} {my_f1_imp:<12.4f} {my_roc_imp:<12.4f}")

print(f"\n{'РЕГРЕССИЯ (Avocado Prices)':-^70}")
print(f"{'Модель':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Бейзлайн (без масштаб.)':<30} {rmse_base:<12.4f} {r2_base:<12.4f}")
print(f"{'Бейзлайн + масштабирование':<30} {rmse_scaled:<12.4f} {r2_scaled:<12.4f}")
print(f"{'+ подбор гиперпараметров':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_rmse:<12.4f} {my_r2:<12.4f}")
print(f"{'Своя улучшенная':<30} {my_rmse_imp:<12.4f} {my_r2_imp:<12.4f}")

ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №1 (KNN)

---------------------КЛАССИФИКАЦИЯ (APS Failure)----------------------
Модель                         F1           ROC-AUC     
------------------------------------------------------
Бейзлайн (без масштаб.)        0.3214       0.8577      
Бейзлайн + масштабирование     0.4528       0.9030      
+ подбор гиперпараметров       0.4839       0.8242      
Своя реализация                0.1463       0.8253      
Своя улучшенная                0.3333       0.7741      

----------------------РЕГРЕССИЯ (Avocado Prices)----------------------
Модель                         RMSE         R²          
------------------------------------------------------
Бейзлайн (без масштаб.)        0.2745       0.5311      
Бейзлайн + масштабирование     0.1572       0.8461      
+ подбор гиперпараметров       0.1516       0.8569      
Своя реализация                0.2169       0.7072      
Своя улучшенная                0.1998       0.7516      
